In [44]:
import requests
from bs4 import BeautifulSoup
import re

import pandas as pd

def recuperer_contenu(url, nom_classe):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Erreur lors de la requête : {e}")
        return []
    
    soup = BeautifulSoup(response.text, 'html.parser')
    balises = soup.find_all(class_=nom_classe)
    
    return balises

In [2]:
# Recuperation des url 

import json
def read_json_file(file_path):
    try:
        # Ouverture et lecture du fichier JSON
        with open(file_path, 'r', encoding='utf-8') as file:
            # Conversion du contenu JSON en dictionnaire Python
            data = json.load(file)
            return data
    except FileNotFoundError:
        print(f"Erreur: Le fichier {file_path} n'existe pas")
        return None
    except json.JSONDecodeError:
        print(f"Erreur: Le fichier {file_path} n'est pas un JSON valide")
        return None
    except Exception as e:
        print(f"Une erreur inattendue s'est produite: {str(e)}")
        return None

def get_ship(data, name):
    return data[name]
    

def get_url(data,name):
    url = get_ship(data,name)['pledge_url']
    return url


data = read_json_file('shipDB_All.json')

# creation d une liste contenant toutes les url 
 
def list_ship(data):
    Flight_ready=[]
    In_Concept = []
    for i in data:
        if 'pledge_url' in data[i].keys():
            Flight_ready.append(i)
        else : 
            In_Concept.append(i)
    return Flight_ready,In_Concept 
    

In [3]:
Flight_ready = [get_ship(data,i)['pledge_url']  for i in list_ship(data)[0]] 

Flight_ready

['https://robertsspaceindustries.com/pledge/ships/origin-100/100i',
 'https://robertsspaceindustries.com/pledge/ships/origin-100/125a',
 'https://robertsspaceindustries.com/pledge/ships/origin-100/135c',
 'https://robertsspaceindustries.com/pledge/ships/origin-300/300i',
 'https://robertsspaceindustries.com/pledge/ships/origin-300/315p',
 'https://robertsspaceindustries.com/pledge/ships/origin-300/325a',
 'https://robertsspaceindustries.com/pledge/ships/origin-300/350r',
 'https://robertsspaceindustries.com/pledge/ships/400i/400i',
 'https://robertsspaceindustries.com/pledge/ships/600i/600i-Explorer',
 'https://robertsspaceindustries.com/pledge/ships/600i/600i-Touring',
 'https://robertsspaceindustries.com/pledge/ships/85x/85X',
 'https://robertsspaceindustries.com/pledge/ships/890-jump/890-Jump',
 'https://robertsspaceindustries.com/pledge/ships/spirit/A1-Spirit',
 'https://robertsspaceindustries.com/pledge/ships/crusader-starlifter/A2-Hercules',
 'https://robertsspaceindustries.com/p

In [4]:
#creation du dataframe

def DataBase():
    nomclasse = "l-sheet__block"
    dict = {}
    liste_dict=[]
    for i in Flight_ready[:]:
        dict = {}
        for e in recuperer_contenu(i, nomclasse):
            e = e.text.replace("\n", "|").replace("\t", "")
            e = e.split("|")
            n = []
            for i in e:
                i = re.sub(r'\s+', ' ', i).strip()
                if len(i)!=0:
                    n.append(i)
            for i in range(0, len(n)-2, 2):
                dict[n[i]] = n[i+1]
        print(dict)
        #dict=fusioncomplete(dict,dict)
        liste_dict.append(dict)
    dict=fusion_complete( liste_dict)

    return dict

DB = DataBase()

Erreur lors de la requête : HTTPSConnectionPool(host='robertsspaceindustries.com', port=443): Max retries exceeded with url: /pledge/ships/origin-100/100i (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1028)')))
{}
Erreur lors de la requête : HTTPSConnectionPool(host='robertsspaceindustries.com', port=443): Max retries exceeded with url: /pledge/ships/origin-100/125a (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1028)')))
{}
Erreur lors de la requête : HTTPSConnectionPool(host='robertsspaceindustries.com', port=443): Max retries exceeded with url: /pledge/ships/origin-100/135c (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self-signed certificate in certificate chain (_ssl.c:1028)')))
{}
Erreur lor

NameError: name 'fusion_complete' is not defined

In [26]:
import pandas as pd
import numpy as np

COLUMNS_TO_DROP = [
    'description', 'pledge_url', 'chassis_id', 'updated_at',
    'production_note', 'loaner', 'id', 'size_class', 'slug',
    'version', 'class_name', 'msrp']
df_stardle = pd.read_json("shipDB_Stardle.json")


# Chargement et nettoyage initial de la base de données des vaisseaux
ship_db = pd.read_json("shipDB_All.json").T
ship_db = ship_db.drop(COLUMNS_TO_DROP, axis=1)

# Extraction des données de base
SIZES = dict(ship_db['sizes'])
SPEED = dict(ship_db['speed'])


def get_speeds(speed_dict=SPEED):
    """
    Extrait les vitesses SCM et maximales des vaisseaux.

    Args:
        speed_dict (dict): Dictionnaire contenant les données de vitesse

    Returns:
        tuple: Deux dictionnaires (vitesses maximales, vitesses SCM)
    """
    scm = {}
    max_speed = {}
    
    for ship, speed_data in speed_dict.items():
        if str(speed_data) == 'None':
            scm[ship] = max_speed[ship] = 'None'
            continue
        
        for speed_type, value in speed_data.items():
            if speed_type == 'scm':
                scm[ship] = value
            elif speed_type == 'max':
                max_speed[ship] = value
                
    return max_speed, scm


def sizes(dimensions_dict=SIZES):
    """
    Extrait les dimensions des vaisseaux.

    Args:
        dimensions_dict (dict): Dictionnaire contenant les données de dimensions

    Returns:
        tuple: Trois dictionnaires (length, beam, height)
    """
    length = {}
    beam = {}
    height = {}

    for ship, dimensions in dimensions_dict.items():
        for dim_type, value in dimensions.items():
            if dim_type == 'length':
                length[ship] = value
            elif dim_type == 'beam':
                beam[ship] = value
            else:
                height[ship] = value
    
    return length, beam, height

In [27]:
   # Création de la base de données finale
stardle_db = pd.read_json('shipDB_Stardle.json').T
stardle_db = stardle_db.drop(['description', 'speed'], axis=1)

    # Ajout des colonnes calculées
max_speeds, scm_speeds = get_speeds()
length_dict, beam_dict, height_dict = sizes()

    # Mise à jour du DataFrame
stardle_db = stardle_db.assign(
        scm=scm_speeds,
        max=max_speeds,
        length=length_dict,
        beam=beam_dict,
        height=height_dict
    )

In [28]:
stardle_db= stardle_db.replace({None: np.nan})
stardle_db['mass'] = stardle_db['mass'].astype(np.float64)

/tmp/ipykernel_9387/3579662187.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  stardle_db= stardle_db.replace({None: np.nan})


In [ ]:
colonnes_numeriques = stardle_db.select_dtypes(include=['int64', 'float64'])

for i in colonnes_numeriques.keys():
    stardle_db[i].replace([np.nan],stardle_db[i].mean(), inplace=True)

/tmp/ipykernel_9387/960163573.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  stardle_db[i].replace([np.nan,],stardle_db[i].mean(), inplace=True)


In [42]:
stardle_db.drop('dimensions', axis=1, inplace=True)

In [43]:
stardle_db.to_csv('stradle_db.csv', index=False)

In [52]:
SW_starships = pd.read_csv("SW_starships.csv")
SW_vehicles = pd.read_csv("SW_vehicles.csv")

In [53]:
SW_starships

,id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,hyperdrive_rating,MGLT,starship_class,pilots,films
0,1,Millennium Falcon,YT-1300 light freighter,Corellian Engineering Corporation,1.000000e+05,34.75,1050.0,4.0,6.0,100000.0,2 months,0.50,75.0,Light freighter,"Han Solo, Chewbacca","A New Hope, The Empire Strikes Back, Return of..."
1,2,X-wing,T-65 X-wing starfighter,Incom Corporation,1.499990e+05,12.50,1050.0,1.0,0.0,110.0,1 week,1.00,100.0,Starfighter,Luke Skywalker,"A New Hope, The Empire Strikes Back, Return of..."
2,3,TIE Fighter,Twin Ion Engine/Ln Starfighter,Sienar Fleet Systems,7.500000e+04,8.99,1200.0,1.0,0.0,65.0,2 days,1.00,100.0,Starfighter,NaN,"A New Hope, The Empire Strikes Back, Return of..."
3,4,Star Destroyer,Imperial I-class Star Destroyer,Kuat Drive Yards,1.500000e+08,1600.00,975.0,47060.0,0.0,36000000.0,2 years,2.00,60.0,Capital ship,NaN,"A New Hope, The Empire Strikes Back, Return of..."
4,5,Slave 1,Firespray-31-class patrol and attack craft,Kuat Systems Engineering,NaN,21.50,1000.0,1.0,6.0,80000.0,1 month,3.00,70.0,Patrol craft,Boba Fett,"The Empire Strikes Back, Attack of the Clones,..."
5,6,A-wing,RZ-1 A-wing interceptor,Alliance Underground Engineering,1.750000e+05,9.60,1300.0,1.0,0.0,40.0,1 week,1.00,120.0,Interceptor,NaN,"Return of the Jedi, The Last Jedi"
6,7,B-wing,A/SF-01 B-wing starfighter,Slayn & Korpil,2.200000e+05,16.90,950.0,1.0,0.0,45.0,1 week,2.00,91.0,Assault starfighter,NaN,Return of the Jedi
7,8,Y-wing,BTL Y-wing starfighter,Koensayr Manufacturing,1.350000e+05,23.40,1000.0,2.0,0.0,110.0,1 week,1.00,80.0,Assault starfighter,NaN,"A New Hope, The Empire Strikes Back, Return of..."
8,9,Executor,Executor-class Star Dreadnought,"Kuat Drive Yards, Fondor Shipyards",1.143350e+09,19000.00,975.0,279144.0,38000.0,250000000.0,6 years,4.00,40.0,Star Dreadnought,Darth Vader,"The Empire Strikes Back, Return of the Jedi"
9,10,Rebel transport,GR-75 medium transport,"Gallofree Yards, Inc.",3.000000e+06,90.00,650.0,6.0,90.0,19000000.0,6 months,4.00,20.0,Medium transport,NaN,"The Empire Strikes Back, Return of the Jedi"
